In [1]:
# you will be prompted with a window asking to grant permissions
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
# fill in the path in your Google Drive in the string below. Note: do not escape slashes or spaces
import os
datadir = "/content/第5章-数据整理"
if not os.path.exists(datadir):
  !ln -s "/content/drive/MyDrive/Python_train/第5章-数据整理" $datadir
os.chdir(datadir)
!pwd

/content/drive/MyDrive/Python_train/第5章-数据整理


# 项目：整理Netflix电影演员评分数据

## 分析目标

此数据分析的目的是，整理不同流派影视作品，比如喜剧片、动作片、科幻片中，各演员出演作品的平均IMDB评分，从而挖掘出各个流派中的高评分作品演员。

本实战项目的目的在于练习整理数据，从而得到可供下一步分析的数据。

## 简介

原始数据集记录了截止至2022年7月美国地区可观看的所有Netflix电视剧及电影数据。数据集包含两个数据表：`titles.csv`和`credits.csv`。

`titles.csv`包含电影及电视剧相关信息，包括影视作品ID、标题、类型、描述、流派、IMDB（一个国外的在线评分网站）评分，等等。`credits.csv`包含超过7万名出现在Netflix影视作品的导演及演员信息，包括名字、影视作品ID、人物名、演职员类型（导演/演员）等。

`titles.csv`每列的含义如下：
- id：影视作品ID。
- title：影视作品标题。
- show_type：作品类型，电视节目或电影。
- description：简短描述。
- release_year：发布年份。
- age_certification：适龄认证。
- runtime：每集电视剧或电影的长度。
- genres：流派类型列表。
- production_countries：出品国家列表。
- seasons：如果是电视剧，则是季数。
- imdb_id：IMDB的ID。
- imdb_score：IMDB的评分。
- imdb_votes：IMDB的投票数。
- tmdb_popularity：TMDB的流行度。
- tmdb_score：TMDB的评分。

`credits.csv`每列的含义如下：
- person_ID：演职员ID。
- id：参与的影视作品ID。
- name：姓名。
- character_name：角色姓名。
- role：演职员类型，演员或导演。

## 1.读取数据

In [4]:
import pandas as pd

original_titles = pd.read_csv("titles.csv")
original_credits = pd.read_csv("credits.csv")

In [5]:
original_titles.sample(5)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
1008,tm175753,Chashme Baddoor,MOVIE,"Siddharth, Omi, and Jai are close friends and ...",2013,NaN,120,"['comedy', 'drama']",['IN'],NaN,tt2229848,5.4,5148.0,2.874,4.5
5145,tm166973,Mixtape,MOVIE,"On the eve of Y2K, orphaned 12-year-old Beverl...",2021,NaN,94,"['comedy', 'family', 'drama', 'romance']",['US'],NaN,tt1587420,6.6,3329.0,16.314,7.1
947,tm182275,Omar,MOVIE,"The drama, the story of three childhood friend...",2013,NaN,96,"['romance', 'thriller', 'drama', 'crime', 'war']",['PS'],NaN,tt2852406,7.5,14318.0,0.600,7.2
2165,tm434917,I Am Jonas,MOVIE,When Jonas was 14 he met the charismatic but m...,2018,NaN,82,"['drama', 'romance', 'european']",['FR'],NaN,tt8168186,7.0,5896.0,8.670,6.8
4697,tm1170388,Persuasion,MOVIE,Living with her snobby family on the brink of ...,2022,PG,107,"['romance', 'drama', 'comedy']","['US', 'GB']",NaN,tt13456318,NaN,NaN,127.205,6.0


In [6]:
original_credits.sample(5)

,person_id,id,name,character,role
8458,48850,tm165593,Christopher Stapleton,Bartender,ACTOR
13339,7277,tm158304,John Hawkes,Roger,ACTOR
22578,746394,ts38731,Jane Herbert,Gretta the Maid,ACTOR
2769,459789,tm191989,Barry Hirschberg,Background (uncredited),ACTOR
7741,21009,tm118637,Philip Craig,Judge K. Ploucha,ACTOR


## 2.评估&清理数据

In [7]:
cleaned_titles = original_titles.copy()
cleaned_credits = original_credits.copy()

### 2.1 titles

#### 2.1.1 结构（整齐度）

In [9]:
cleaned_titles.head(5)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


从数据的部分10行来看，`cleaned_titles`里的`genres`和`production_countries`的变量中包含多个值，应当进行拆分。

先提取任意一个`genres`变量的值进行观察。

In [10]:
cleaned_titles["genres"][1]

"['drama', 'crime']"

虽然`genres`表示形式是列表，但其实际类型并非字符串列表，而是字符串，无法直接用`value_counts`统计各个值出现的次数。
我们可以使用Python内置的`eval`函数，它可以把字符串转换成表达式，所以可以帮我们把表示列表的字符串转换成列表本身。

In [11]:
cleaned_titles['genres'] = cleaned_titles['genres'].apply(lambda s: eval(s))
cleaned_titles['genres'][1]

['drama', 'crime']

转换为列表后，就能用DataFrame的`explode`方法，把那个列的列表值拆分成单独的行。

In [13]:
cleaned_titles = cleaned_titles.explode("genres")
cleaned_titles.head(3)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,documentation,['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,crime,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179


接下来，针对`production_countries`列也是一样的流程。

每个观察值的`production_countries`值并不表示单个流派，而是一系列流派。先提取任意一个`production_countries`变量的值进行观察。

In [14]:
cleaned_titles["production_countries"][1]

,production_countries
1,['US']
1,['US']


可以看到，`production_countries`也是一样的问题，虽然表示形式是列表，但其实际类型并非字符串列表，而是字符串，难以进行拆分。
我们可以再次利用`eval`函数进行类型转换，并检查转换后确实是列表类型。

In [15]:
cleaned_titles['production_countries'] = cleaned_titles['production_countries'].apply(lambda s: eval(s))
cleaned_titles['production_countries'][0]

['US']

确认类型转换完毕后，还是用`explode`方法，把列表值拆分成单独的行。

In [16]:
cleaned_titles = cleaned_titles.explode('production_countries')
cleaned_titles.sample(3)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
1963,tm211554,Beyond Skyline,MOVIE,Detective Mark Corley storms his way onto an a...,2017,R,105,action,US,NaN,tt6856700,5.3,21834.0,10.535,5.200
501,tm160596,Zozo,MOVIE,A Lebanese boy gets separated from his family ...,2005,NaN,105,european,CZ,NaN,tt0448267,6.5,3225.0,1.899,6.000
4953,ts312071,The Way of the Househusband,SHOW,"Tatsu, a notorious and feared yakuza leader ni...",2021,TV-MA,18,romance,JP,1.0,tt13357124,7.3,3724.0,20.159,8.591


#### 2.1.2 内容（干净度）

##### 整体观察

In [18]:
cleaned_titles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17818 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    17818 non-null  object 
 1   title                 17817 non-null  object 
 2   type                  17818 non-null  object 
 3   description           17790 non-null  object 
 4   release_year          17818 non-null  int64  
 5   age_certification     10889 non-null  object 
 6   runtime               17818 non-null  int64  
 7   genres                17755 non-null  object 
 8   production_countries  17439 non-null  object 
 9   seasons               6224 non-null   float64
 10  imdb_id               17116 non-null  object 
 11  imdb_score            16976 non-null  float64
 12  imdb_votes            16945 non-null  float64
 13  tmdb_popularity       17663 non-null  float64
 14  tmdb_score            17241 non-null  float64
dtypes: float64(5), int64(2), 

从输出结果来看，`cleaned_titles`数据共有17818条观察值，`title`、`description`、`age_certification`、`genres`、`production_countries`、`seasons`、`imdb_id`、`imdb_score`、`tmdb_popularity`、`tmdb_score`、`imdb_votes`、`tmdb_popularity`、`tmdb_score`变量均存在缺失值，将在后续进行评估和清理。

此外，`release_year`表示年份，数据类型不应为数字，应为日期，所以需要进行数据格式转换。

##### 数据类型转换

In [21]:
cleaned_titles["release_year"] = pd.to_datetime(cleaned_titles["release_year"],format='%Y')
cleaned_titles["release_year"].sample(3)

,release_year
3759,2019-01-01
3559,2020-01-01
36,1990-01-01


##### 缺失值处理

在`cleaned_titles`中，`title`、`description`、`age_certification`、`genres`、`production_countries`、`seasons`、`imdb_id`、`imdb_score`、`tmdb_popularity`、`tmdb_score`、`imdb_votes`、`tmdb_popularity`、`tmdb_score`变量存在缺失值。

由于影视作品的标题、描述、适龄认证、发行国家、电视剧季数、IMDB的ID、TMDB的流行度、TMDB的评分，并不影响我们挖掘各个流派中的高IMDB评分作品演员，所以可以保留`title`、`description`、`age_certification`、`production_countries`、`seasons`、`imdb_id`、`tmdb_popularity`、`tmdb_score`、`imdb_votes`、`tmdb_popularity`、`tmdb_score`变量值存在空缺的观察值。

但`imdb_score`和`genres`，即IMDB评分和流派，和我们后续要做的分析息息相关。

先提取出`imdb_score`缺失观察值进行查看。

In [23]:
cleaned_titles.query("imdb_score.isnull()")

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945-01-01,TV-MA,51,documentation,US,1.0,NaN,NaN,NaN,0.600,NaN
75,tm132164,Bill Hicks: Sane Man,MOVIE,Sane Man was filmed before Bill recorded ‘Dang...,1989-01-01,R,80,comedy,US,NaN,NaN,NaN,NaN,3.377,7.5
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991-01-01,TV-G,18,documentation,JP,12.0,NaN,NaN,NaN,7.730,7.8
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991-01-01,TV-G,18,family,JP,12.0,NaN,NaN,NaN,7.730,7.8
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991-01-01,TV-G,18,reality,JP,12.0,NaN,NaN,NaN,7.730,7.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5810,tm1225897,Social Man,MOVIE,Two competitive social media Influencers go he...,2021-01-01,NaN,96,drama,NaN,NaN,tt20198164,NaN,NaN,NaN,NaN
5833,ts307884,HQ Barbers,SHOW,When a family run barber shop in the heart of ...,2021-01-01,TV-14,24,comedy,NG,1.0,NaN,NaN,NaN,0.840,NaN
5840,tm1216735,Sun of the Soil,MOVIE,"In 14th-century Mali, an ambitious young royal...",2022-01-01,NaN,26,NaN,NaN,NaN,NaN,NaN,NaN,1.179,7.0
5844,tm1074617,Bling Empire - The Afterparty,MOVIE,"The stars of ""Bling Empire"" discuss the show's...",2021-01-01,NaN,35,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN


由于缺失分析所需的核心数据`imdb_score`，我们将把这些观察值删除，并查看删除后该列空缺值个数和。

In [24]:
cleaned_titles = cleaned_titles.dropna(subset=["imdb_score"])
cleaned_titles["imdb_score"].isnull().sum()

np.int64(0)

提取出`genres`缺失观察值进行查看。

In [25]:
cleaned_titles.query("genres.isnull()")

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
1813,ts77824,My Next Guest Needs No Introduction With David...,SHOW,TV legend David Letterman teams up with fascin...,2018-01-01,TV-MA,50,NaN,US,4.0,tt7829834,7.8,5581.0,8.217,7.6
1939,ts215037,Minecraft: Story Mode,SHOW,"MInecraft: Story Mode is an interactive, anima...",2018-01-01,TV-PG,52,NaN,US,1.0,tt10498322,5.6,347.0,NaN,NaN
2386,ts74805,A Little Help with Carol Burnett,SHOW,In this unscripted series starring comedy lege...,2018-01-01,TV-G,24,NaN,US,1.0,tt7204366,6.3,237.0,1.621,6.2
2658,ts265844,#ABtalks,SHOW,#ABtalks is a YouTube interview show hosted by...,2018-01-01,TV-PG,68,NaN,NaN,1.0,tt12635254,9.6,7.0,NaN,NaN
4274,tm1172010,The Lockdown Plan,MOVIE,NaN,2020-01-01,NaN,49,NaN,NaN,NaN,tt13079112,6.5,NaN,NaN,NaN
4648,tm1113921,In Vitro,MOVIE,'In Vitro' is an otherworldly rumination on me...,2019-01-01,NaN,27,NaN,NaN,NaN,tt10545994,7.7,NaN,NaN,NaN


由于缺失分析所需的核心数据`genres`，我们将把这些观察值删除，并查看删除后该列空缺值个数和。

In [26]:
cleaned_titles = cleaned_titles.dropna(subset=["genres"])
cleaned_titles["genres"].isnull().sum()

np.int64(0)

##### 重复值处理

根据数据变量的含义以及内容来看，`cleaned_titles`里不应该存在每个变量值都相同的观察值，因此查看是否存在重复值。

In [28]:
cleaned_titles.duplicated().sum()

np.int64(0)

输出结果为0，说明不存在重复值。

##### 不一致数据处理

针对`cleaned_titles`，不一致数据可能存在于`genres`和`character`变量中，我们将查看是否存在多个不同值指代同一流派，以及多个不同值指代同一国家的情况。

In [30]:
cleaned_titles["genres"].value_counts()

,count
genres,
drama,3357
comedy,2419
thriller,1446
action,1339
romance,1080
crime,1066
documentation,981
family,769
animation,732


从上面看出，`genres`列里并不存在不一致数据，各个值都在指代不同的流派。但是里面还存在空字符串表示的流派，并非有效数据，因此可以进行删除。

删除后，查看`cleaned_titles`里是否还存在`genres`为空字符串的行：

In [32]:
cleaned_titles = cleaned_titles.query("genres != ''")
cleaned_titles.query("genres == ''")

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score


接下来，针对`production_countries`列也是一样的流程，利用`value_counts`方法，得到`production_countries`的列表里面各个值的出现次数。

In [33]:
cleaned_titles["production_countries"].value_counts()

,count
production_countries,
US,5648
IN,1610
GB,1068
JP,1046
FR,720
...,...
CU,1
LK,1
GT,1


由于`value_counts`执行结果中有太多值，Pandas只会默认显示开头和结尾的一些值。要完整展示结果，可以把`display.max_rows`设置为`None`，即取消展示行数上限。

但因为我们只是在当前调用`value_counts`时才需要看完整结果，所以可以结合`option_context`，只更改临时上限。

In [34]:
with pd.option_context('display.max_rows', None):
    print(cleaned_titles['production_countries'].value_counts())

production_countries
US         5648
IN         1610
GB         1068
JP         1046
FR          720
ES          637
KR          637
CA          608
DE          383
CN          295
MX          264
IT          224
BR          221
AU          217
TR          195
PH          192
AR          150
ID          149
BE          148
TW          133
NG          131
PL          126
ZA          103
HK          102
NL          102
CO           94
EG           93
DK           89
TH           87
SE           81
LB           70
NO           68
AE           52
IE           49
SG           47
XX           43
IL           42
RU           41
CL           35
CH           33
PS           32
BG           31
MY           30
SA           28
IS           28
AT           28
NZ           27
LU           27
PE           26
RO           25
QA           24
CZ           22
JO           19
FI           18
HU           18
UY           15
MA           15
PT           14
KH           10
KW           10
PR            9
PK 

从以上输出结果来看，出品国家都用两位的国家代码来表示，除了里面存在一个的`Lebanon`值。

`Lebanon`的国家代码是`LB`，出现了39次，说明此处数据不一致。`LB`和`Lebanon`都在表示同一国家，需要进行统一。


把`cleaned_titles`里，`production_countries`的`"LB"`和`"Lebanon"`统一为`LB`，并检查替换后是否还存在`"LB"`：

In [35]:
cleaned_titles["production_countries"] = cleaned_titles['production_countries'].replace({"Lebanon":"LB"})

with pd.option_context('display.max_rows', None):
    print(cleaned_titles.explode('production_countries')['production_countries'].value_counts())

production_countries
US    5648
IN    1610
GB    1068
JP    1046
FR     720
ES     637
KR     637
CA     608
DE     383
CN     295
MX     264
IT     224
BR     221
AU     217
TR     195
PH     192
AR     150
ID     149
BE     148
TW     133
NG     131
PL     126
ZA     103
HK     102
NL     102
CO      94
EG      93
DK      89
TH      87
SE      81
LB      71
NO      68
AE      52
IE      49
SG      47
XX      43
IL      42
RU      41
CL      35
CH      33
PS      32
BG      31
MY      30
SA      28
AT      28
IS      28
NZ      27
LU      27
PE      26
RO      25
QA      24
CZ      22
JO      19
FI      18
HU      18
MA      15
UY      15
PT      14
KW      10
KH      10
PK       9
PR       9
UA       8
VN       8
MT       8
SU       7
CD       7
TN       7
LT       7
IR       7
GH       6
SN       6
AL       6
KE       6
IQ       5
MU       5
CY       5
TZ       4
SY       4
MC       4
IO       4
KN       4
GR       4
BD       3
BS       3
DZ       3
GL       3
AO       3
CM       3


另外，里面还可能存在空字符串表示的国家代码，并非有效数据。但由于出品国家并非分析所需的关键信息，所以可以保留出品国家为空的观察值。

##### 无效/错误数据处理

可以通过DataFrame的`describe`方法，对数值统计信息进行快速了解。

In [44]:
cleaned_titles.describe()

,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
count,16970,16970.000000,5954.000000,16970.000000,1.694100e+04,16842.000000,16515.000000
mean,2015-11-14 22:42:51.974072064,80.912552,2.455492,6.514207,3.281655e+04,29.396307,6.846933
min,1954-01-01 00:00:00,0.000000,1.000000,1.500000,5.000000e+00,0.600000,1.000000
25%,2015-01-01 00:00:00,45.000000,1.000000,5.800000,7.800000e+02,4.070000,6.200000
50%,2018-01-01 00:00:00,90.000000,2.000000,6.600000,3.508000e+03,10.195000,6.900000
75%,2020-01-01 00:00:00,107.000000,3.000000,7.300000,1.697800e+04,23.639000,7.500000
max,2022-01-01 00:00:00,225.000000,42.000000,9.500000,2.294231e+06,2274.044000,10.000000
std,NaN,39.596172,2.869428,1.131095,1.141492e+05,93.178235,1.078831


从以上统计信息来看，`cleaned_titles`里不存在脱离现实意义的数值。

### 2.2 credits

#### 2.2.1 结构（整齐度）

In [17]:
cleaned_credits.head(5)

,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR


从头部的5行数据来看，`cleaned_credits`数据符合“每个变量为一列，每个观察值为一行，每种类型的观察单位为一个表格”，因此不存在结构性问题。

#### 2.2.2 内容（干净度）

##### 整体观察

In [27]:
cleaned_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77801 entries, 0 to 77800
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  77801 non-null  int64 
 1   id         77801 non-null  object
 2   name       77801 non-null  object
 3   character  68029 non-null  object
 4   role       77801 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.0+ MB


从输出结果来看，`cleaned_credits`数据共有77801条观察值，其中`character`变量存在缺失值，将在后续进行评估和清理。

此外，`person_id`表示演职员ID，数据类型不应为数字，应为字符串，所以需要进行数据格式转换。

##### 数据类型转化

In [45]:
cleaned_credits["person_id"] = cleaned_credits["person_id"].astype("str")
cleaned_credits["person_id"]

,person_id
0,3748
1,14658
2,7064
3,3739
4,48933
...,...
77796,736339
77797,399499
77798,373198
77799,378132


##### 缺失值处理

评估`cleaned_credits`的缺失数据，其中只有`character`变量存在缺失值。

角色名并不影响我们挖掘各个流派中的高IMDB评分作品演员，并且此变量缺失也有可能因为演职员类别是导演，没有对应角色，因此可以保留`character`变量值存在空缺的观察值。

##### 重复值处理

查看`cleaned_credits`数据表是否存在重复值。

In [29]:
cleaned_credits.duplicated().sum()

np.int64(0)

输出结果为0，说明不存在重复值。

##### 不一致数据处理

针对`original_credits`，不一致数据可能存在于`role`中，我们将查看是否存在多个不同值指代同一演职员类型的情况。

In [36]:
cleaned_credits['role'].value_counts()

,count
role,
ACTOR,73251
DIRECTOR,4550


从以上输出结果来看，`role`只有两种可能的值，`ACTOR`或`DIRECTOR`，不存在不一致数据。我们可以把这列的类型转换为`Category`，好处是比字符串类型更节约内存空间，也能表明说值的类型有限。

In [43]:
cleaned_credits["role"] = cleaned_credits["role"].astype("category")
cleaned_credits["role"]

,role
0,ACTOR
1,ACTOR
2,ACTOR
3,ACTOR
4,ACTOR
...,...
77796,ACTOR
77797,ACTOR
77798,ACTOR
77799,ACTOR


##### 无效/错误数据处理

In [46]:
cleaned_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77801 entries, 0 to 77800
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   person_id  77801 non-null  object  
 1   id         77801 non-null  object  
 2   name       77801 non-null  object  
 3   character  68029 non-null  object  
 4   role       77801 non-null  category
dtypes: category(1), object(4)
memory usage: 2.4+ MB


`cleaned_credits`由于不包含表示数值含义的变量，因此无需用`describe`检查。

## 3.整理数据

In [47]:
cleaned_titles.sample(5)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
5072,tm1131061,Hilda and the Mountain King,MOVIE,"When Hilda wakes up in the body of a troll, sh...",2021-01-01,NaN,85,animation,CA,NaN,tt15777864,7.7,2408.0,67.409,7.6
3434,ts223344,The Gift,SHOW,A painter in Istanbul embarks on a personal jo...,2019-01-01,TV-MA,43,fantasy,TR,3.0,tt10075318,7.0,21802.0,16.327,7.7
5407,tm1129394,The Princess Switch 3: Romancing the Star,MOVIE,A priceless relic is stolen from identical roy...,2021-01-01,PG,106,drama,US,NaN,tt14731142,5.5,5728.0,39.432,6.9
1269,tm228677,The Space Between Us,MOVIE,"In this interplanetary adventure, a space shut...",2016-01-01,PG-13,121,romance,CN,NaN,tt3922818,6.4,52698.0,23.219,7.1
3571,tm493511,The Sky Is Pink,MOVIE,Based on the love story of a couple spanning 2...,2019-01-01,NaN,143,drama,IN,NaN,tt8902990,7.5,8664.0,6.395,7.8


In [48]:
cleaned_credits.sample(5)

,person_id,id,name,character,role
14746,1344,tm177258,Vir Das,Arun,ACTOR
15727,139879,tm34646,Gardo Versoza,NaN,ACTOR
59604,589479,tm827640,Kylie Jenner,Self,ACTOR
60215,1398046,ts225377,Anshul Chauhan,Rashmi Malik,ACTOR
60517,104987,tm859143,Girish Malik,NaN,DIRECTOR


对数据的整理，与分析方向紧密相关。此次数据分析目标是，整理不同流派影视作品，比如喜剧片、动作片、科幻片中，演员出演作品的平均IMDB评分，从而挖掘出各个流派中的高评分作品演员。

那为了能同时获得流派与演员数据，我们需要把`cleaned_titles`和`cleaned_credits`，通过`id`作为键进行连接，因为两个数据表中`id`都是影视作品ID。

In [52]:
credits_with_titles = pd.merge(cleaned_titles,cleaned_credits,on="id",how="inner")
credits_with_titles.sample(3)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,person_id,name,character,role
136595,ts78217,Welcome to the Family,SHOW,"You don't choose family, family chooses you.",2018-01-01,TV-MA,48,comedy,ES,2.0,tt7490342,6.6,455.0,5.935,7.4,2201137,Nonna Cardoner,Sara,ACTOR
256968,tm1177856,Without Saying Goodbye,MOVIE,The opposite lives of a workaholic architect a...,2022-01-01,NaN,96,comedy,ES,NaN,tt14404596,5.7,2446.0,93.966,7.2,396057,Marta Tébar,NaN,ACTOR
60633,tm38013,Players,MOVIE,Learning of a fortune in gold being secretly s...,2012-01-01,NaN,165,action,IN,NaN,tt1784589,4.2,4534.0,4.977,5.2,63055,Neil Nitin Mukesh,Spider,ACTOR


由于我们只对挖掘演员的参演作品口碑感兴趣，导演不在我们的分析范围内，因此根据`role`，筛选出类型为`ACTOR`的观察值，供后续分析。

In [53]:
actor_with_titles = credits_with_titles.query("role == 'ACTOR'")
actor_with_titles.sample(3)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,person_id,name,character,role
185257,tm471967,Lost Girls,MOVIE,"When Mari Gilbert's daughter disappears, polic...",2020-01-01,R,95,crime,US,NaN,tt3111426,6.1,21588.0,12.547,6.1,1228086,Harvey B. Jackson,Scott's Friend,ACTOR
271154,tm1143058,How I Fell in Love with a Gangster,MOVIE,A mysterious woman recounts the rise and fall ...,2022-01-01,NaN,179,crime,PL,NaN,tt16453244,6.0,2248.0,29.595,6.7,1562710,Ewelina Pankowska,Gabrysia,ACTOR
240895,tm456513,Army of the Dead,MOVIE,"Following a zombie outbreak in Las Vegas, a gr...",2021-01-01,R,148,action,US,NaN,tt0993840,5.7,169348.0,131.869,6.3,11208,Nora Arnezeder,Lilly / The Coyote,ACTOR


为了挖掘出各个流派中的高IMDB评分作品演员，我们需要先根据**流派**和**演员**进行分组。

对演员进行分组的时候，选择的是用`person_id`而不是`name`变量，原因是名字容易出现错拼或者重名的情况，演职员ID会比演员姓名更加准确地反映是哪位演员。

分组后，我们只需要对`imdb_score`的值进行聚合计算，因此只提取`imdb_score`变量，然后调用`mean`，来计算各个流派影视作品中，每位演员参演作品的平均IMDB评分。

In [54]:
imdb_score_groupby_genres_and_person_id = actor_with_titles.groupby(["genres", "person_id"])["imdb_score"].mean()
imdb_score_groupby_genres_and_person_id

genres   person_id
action   1000         6.866667
         100007       7.000000
         100013       6.400000
         100019       6.500000
         100020       6.500000
                        ...   
western  993735       6.500000
         998673       7.300000
         998674       7.300000
         998675       7.300000
         99940        4.000000
Name: imdb_score, Length: 168881, dtype: float64

我们可以调用`reset_index`，对层次化索引进行重置，得到更加规整的DataFrame。

In [55]:
imdb_score_groupby_genres_and_person_id_df = imdb_score_groupby_genres_and_person_id.reset_index()
imdb_score_groupby_genres_and_person_id_df

,genres,person_id,imdb_score
0,action,1000,6.866667
1,action,100007,7.000000
2,action,100013,6.400000
3,action,100019,6.500000
4,action,100020,6.500000
...,...,...,...
168876,western,993735,6.500000
168877,western,998673,7.300000
168878,western,998674,7.300000
168879,western,998675,7.300000


现在针对流派和演员分组的IMDB评分数据已经整理好，可以进入后续的分析步骤了。

但我们当前可以继续做一些数据整理，比如对上面的结果再次进行分组，找出各个流派里演员作品最高的平均评分是多少、最高评分对应的演员名字是什么。

要得到这一结果，我们需要再次用`genres`进行分组，然后提取出`imdb_score`变量，计算其最大值。

In [56]:
genres_max_scores = imdb_score_groupby_genres_and_person_id_df.groupby("genres")["imdb_score"].max()
genres_max_scores

,imdb_score
genres,
action,9.3
animation,9.3
comedy,9.2
crime,9.5
documentation,9.1
drama,9.5
european,8.9
family,9.3
fantasy,9.3


In [57]:
genres_max_scores_with_person_id = pd.merge(imdb_score_groupby_genres_and_person_id_df,genres_max_scores,on=["genres","imdb_score"])
genres_max_scores_with_person_id

,genres,person_id,imdb_score
0,action,12790,9.3
1,action,1303,9.3
2,action,21033,9.3
3,action,336830,9.3
4,action,86591,9.3
...,...,...,...
131,war,826547,8.8
132,western,22311,8.9
133,western,28166,8.9
134,western,28180,8.9


从以上结果可以看出，最高分对应的演员不一定只有一位，可能有多位演员的平均得分相同。

为了得到演员ID所对应的演员名字，我们可以和`cleaned_credits`这个DataFrame进行连接。这个DataFrame还有其它列，我们只需要得到`person_id`和`name`的对应，所以可以先提取出那两列，并把重复行删除。

In [58]:
actor_id_with_names = cleaned_credits[["person_id","name"]].drop_duplicates()
actor_id_with_names.sample(3)

,person_id,name
52805,305122,Jack Yao
35650,885190,Constantin Pappas
73220,250167,Reno Selmser


下一步就可以把`actor_id_with_names`与前面得到的`genres_max_score_with_person_id`进行连接，增加`name`变量，从而展示平均评分最高的演员名字。

In [59]:
genres_max_score_with_actor_name = pd.merge(genres_max_scores_with_person_id,actor_id_with_names,on="person_id")
genres_max_score_with_actor_name

,genres,person_id,imdb_score,name
0,action,12790,9.3,Olivia Hack
1,action,1303,9.3,Jessie Flower
2,action,21033,9.3,Zach Tyler
3,action,336830,9.3,André Sogliuzzo
4,action,86591,9.3,Cricket Leigh
...,...,...,...,...
131,war,826547,8.8,Yuto Uemura
132,western,22311,8.9,Koichi Yamadera
133,western,28166,8.9,Megumi Hayashibara
134,western,28180,8.9,Unsho Ishizuka


为了把相同流派都排序在一起，我们还可以用`sort_values`方法，把结果里面的行根据`genres`进行排序，然后用`reset_index`把索引重新排序。

索引重新排序后，DataFrame会多出`index`一列，我们可以再把`index`列进行删除。

In [65]:
genres_max_score_with_actor_name = genres_max_score_with_actor_name.sort_values("genres").reset_index().drop("index",axis=1)
genres_max_score_with_actor_name

,genres,person_id,imdb_score,name
0,action,12790,9.3,Olivia Hack
1,action,1303,9.3,Jessie Flower
2,action,21033,9.3,Zach Tyler
3,action,336830,9.3,André Sogliuzzo
4,action,86591,9.3,Cricket Leigh
...,...,...,...,...
131,war,826547,8.8,Yuto Uemura
132,western,28180,8.9,Unsho Ishizuka
133,western,22311,8.9,Koichi Yamadera
134,western,28166,8.9,Megumi Hayashibara
